In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from collections import defaultdict
import traceback

import json

In [2]:
linux = True

In [3]:
# the source is here:https://docs.google.com/spreadsheets/d/1l8gf56ifdBwAptohozsdVidx9dUS-yNx2qxamYsooTw/edit?usp=sharing
if linux: # should move to s3
    file_path = '/home/ec2-user/data/datasets_feature_sets_v1-1.csv'
    df_feature_set = pd.read_csv(file_path)
else: 
    df_feature_set = pd.read_csv('/home/jovyan/projects/data/datasets_feature_sets_v1-1.csv')

In [4]:
df_feature_set[['dataset_id', 'dataset']] = df_feature_set[['dataset_id', 'dataset']].ffill()

In [5]:
df_feature_set.head()

,dataset_id,dataset,feature_set,status,current_prod _path,pheno-master-data-collection-v1-prod-us-east-1_relative_path,pheno-big-data-prod-us-east-1_relative_path,contains_sensitive,notes,count_parquet_relative_path,count_field_name,Unnamed: 11,Unnamed: 12,Seed_july,MBZUAI_july,UCSF_july,Cornell_july,Harvard- Joslin Diabetes Center,academic_f&f_july
0,0.0,population,population,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/popula...,/population/population.parquet,NaN,NaN,NaN,/population/population.parquet,sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1.0,events,events,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/events...,/events/events.parquet,NaN,NaN,NaN,/events/events.parquet,research_stage_date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2.0,anthropometrics,anthropometrics,READY on v1.1 Master bucket,s3://pheno-anthropometrics-derived-prod/v1.1/a...,/anthropometrics/anthropometrics.parquet,NaN,NaN,NaN,/anthropometrics/anthropometrics.parquet,bmi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,3.0,fundus,fundus,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/fun...,/fundus/fundus.parquet,NaN,NaN,NaN,/fundus/fundus.parquet,"fundus_image_left, fundus_image_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,3.0,fundus,microvasculature,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/mic...,/fundus/microvasculature.parquet,NaN,NaN,NaN,/fundus/microvasculature.parquet,"vessel_density_left, vessel_density_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
df_feature_set_ready = df_feature_set.query('status == "READY on v1.1 Master bucket"')
df_feature_set_ready.head()

,dataset_id,dataset,feature_set,status,current_prod _path,pheno-master-data-collection-v1-prod-us-east-1_relative_path,pheno-big-data-prod-us-east-1_relative_path,contains_sensitive,notes,count_parquet_relative_path,count_field_name,Unnamed: 11,Unnamed: 12,Seed_july,MBZUAI_july,UCSF_july,Cornell_july,Harvard- Joslin Diabetes Center,academic_f&f_july
0,0.0,population,population,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/popula...,/population/population.parquet,NaN,NaN,NaN,/population/population.parquet,sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1.0,events,events,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/events...,/events/events.parquet,NaN,NaN,NaN,/events/events.parquet,research_stage_date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2.0,anthropometrics,anthropometrics,READY on v1.1 Master bucket,s3://pheno-anthropometrics-derived-prod/v1.1/a...,/anthropometrics/anthropometrics.parquet,NaN,NaN,NaN,/anthropometrics/anthropometrics.parquet,bmi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,3.0,fundus,fundus,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/fun...,/fundus/fundus.parquet,NaN,NaN,NaN,/fundus/fundus.parquet,"fundus_image_left, fundus_image_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,3.0,fundus,microvasculature,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/mic...,/fundus/microvasculature.parquet,NaN,NaN,NaN,/fundus/microvasculature.parquet,"vessel_density_left, vessel_density_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [7]:
master_path = "s3://pheno-master-data-collection-v1-prod-us-east-1"

In [8]:
print(df_feature_set_ready.shape)
# none = [np.nan, pd.na]
df = df_feature_set_ready.query('@pd.notna(count_parquet_relative_path) and @pd.notna(count_field_name)') # in @none
df.shape

(60, 19)


(52, 19)

In [9]:
print(df.shape)
df_no_dup = df.drop_duplicates(['dataset', 'feature_set', 'count_parquet_relative_path', 'count_field_name'])
print(df_no_dup.shape)

(52, 19)
(52, 19)


In [13]:
count_dict = defaultdict(dict)
for index, row in df_no_dup.iterrows():
    try:
        # if row['dataset'] == 'diet_logging':
        #     continue
        dataset = row['dataset']
        feature_set = row['feature_set']
        
        #read parquet file
        full_path = f"{master_path}{row['count_parquet_relative_path']}"
        df_parquet = pd.read_parquet(full_path).reset_index()
        
        if dataset=='medications':
            row['count_field_name'] = 'medication'
        res_dict =  {
            "df_parquet_shape": df_parquet.shape[0],
            "count_field_name": row['count_field_name'],
            "count_parquet_relative_path": row['count_parquet_relative_path'],
            "particioant_id_nunique": df_parquet['participant_id'].nunique()
            }
        
        max_col = ''
        max_count = 0
        
        for col in row['count_field_name'].split(','):
            # remove white space
            col = col.strip()
            if col not in df_parquet.columns:
                print (f"ERROR: {col} not in {full_path} : {df_parquet.columns}")
                
            no_na_df = df_parquet.dropna(subset=[col]).copy()
            
            if dataset != 'events': 
                date_col = 'collection_date'
            else: 
                date_col = 'research_stage_date'
                
            if date_col not in no_na_df.columns:
                date_col = 'collection_timestamp'
            if date_col not in no_na_df.columns:
                print (f"ERROR: {date_col} not in {full_path}")
                first_date = None
                last_date = None
            else:   
                first_date = no_na_df[date_col].min() # first date
                last_date = no_na_df[date_col].max() # last date
            
            nunique_participants_not_na = no_na_df.participant_id.nunique() # number of unique participants
            count_col_notna = no_na_df.shape[0] # number of not na entries
            
            if no_na_df.shape[0] > max_count:
                max_count = no_na_df.shape[0]
                max_col = col
                
                
            res_dict[col] = {
                "count_col_notna": count_col_notna,  
                "nunique_participants_not_na" : nunique_participants_not_na,
                "first_date": first_date,
                "last_date": last_date,
            }
        del df_parquet
        res_dict['max_col'] = max_col
        count_dict[f"{int(row['dataset_id'])}-{dataset}"][feature_set] = res_dict
        del no_na_df
        
        
    except Exception as e: 
        print(row['dataset'], full_path)
        print(traceback.format_exc())
    

# with open('count_dict.json', 'w') as f:
#             json.dump(count_dict, f)

ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/population/population.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/fundus/microvasculature.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/fundus/microvasculature.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/hrv.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/hrv.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/hrv.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/hrv.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/raw/hrv.parquet
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/sleep/raw/hrv.parquet
ERROR: 

In [14]:
count_dict['17-cgm']['cgm']

{'df_parquet_shape': 10087,
 'count_field_name': 'cgm_time_series',
 'count_parquet_relative_path': '/cgm/cgm.parquet',
 'particioant_id_nunique': 10011,
 'cgm_time_series': {'count_col_notna': 10087,
  'nunique_participants_not_na': 10011,
  'first_date': Timestamp('2019-01-07 00:14:00+0200', tz='Asia/Jerusalem'),
  'last_date': Timestamp('2023-03-13 00:08:00+0200', tz='Asia/Jerusalem')},
 'max_col': 'cgm_time_series'}

In [15]:

full_data = list()
for k, v in count_dict.items():
    for kk, vv in v.items():
        data = list()
        data.append(k)
        data.append(kk)
        col = vv['max_col']
        data.append(vv[col]["count_col_notna"])
        data.append(vv[col]["nunique_participants_not_na"])
        data.append(vv[col]["first_date"])
        data.append(vv[col]["last_date"])
        full_data.append(data)
        


In [16]:
df_res = pd.DataFrame(full_data)
df_res.columns = ['dataset', 'feature_set', 'count_col_notna', 'nunique_participants_not_na', 'first_date', 'last_date']
df_res.head()


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
0,0-population,population,11179,11179,None,None
1,1-events,events,23300,11179,2018-12-05 00:00:00,2023-03-19 00:00:00
2,2-anthropometrics,anthropometrics,13568,10781,2018-11-22 00:00:00,2023-02-07 00:00:00
3,3-fundus,fundus,7079,7049,2021-02-17 00:00:00,2022-12-04 00:00:00
4,3-fundus,microvasculature,7079,7049,None,None


In [44]:
dict_dataset = {
    'population': 'events',
    'microvasculature': 'fundus',
    'hrv': 'sleep',
    'hrv_raw': 'sleep',
    'pca': 'human_genetics',
    'iglu': 'iglu_daily'
}
for k, v in dict_dataset.items():
    df_res.loc[df_res['feature_set']== k, 'first_date'] =  df_res.loc[df_res['feature_set']== v, 'first_date'].values
    df_res.loc[df_res['feature_set']== k, 'last_date'] = df_res.loc[df_res['feature_set']== v, 'last_date'].values

df_res.head()

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
0,0-population,population,11179,11179,2018-12-05 00:00:00,2023-03-19 00:00:00
1,1-events,events,23300,11179,2018-12-05 00:00:00,2023-03-19 00:00:00
2,2-anthropometrics,anthropometrics,13568,10781,2018-11-22 00:00:00,2023-02-07 00:00:00
3,3-fundus,fundus,7079,7049,2021-02-17 00:00:00,2022-12-04 00:00:00
4,3-fundus,microvasculature,7079,7049,2021-02-17 00:00:00,2022-12-04 00:00:00


In [18]:
# df_res.to_csv('s3://ds-users/mariag/pheno_master_data_collection_v1_1_count.csv', index=False)

In [46]:
df_res.query('dataset == "17-cgm"')

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
46,17-cgm,cgm,10087,10011,2019-01-07 00:14:00+02:00,2023-03-13 00:08:00+02:00
47,17-cgm,timeseries,10087,10011,2019-01-07 00:14:00+02:00,2023-03-13 00:08:00+02:00
48,17-cgm,iglu,10087,10011,2019-01-07 00:00:00,2023-03-24 00:00:00
49,17-cgm,iglu_daily,112023,10011,2019-01-07 00:00:00,2023-03-24 00:00:00


In [47]:
import datetime

# Create a function to convert the date string to a datetime object
def convert_date_string(date_string):
    if date_string is None or date_string == 'None':
        return None
    else:
        date_string_without_timezone = date_string[:19]
        datetime_object = datetime.datetime.strptime(date_string_without_timezone, '%Y-%m-%d %H:%M:%S')
        date = datetime_object.date()
        return date
df_convert = df_res.copy()


df_convert["first_date"] = df_convert["first_date"].apply(str)
df_convert["last_date"] = df_convert["last_date"].apply(str)
# Convert all the dates in the DataFrame to datetime objects
df_convert["first_date"] = df_convert["first_date"].apply(convert_date_string)
df_convert["last_date"] = df_convert["last_date"].apply(convert_date_string)




In [48]:
df_convert.query('dataset == "17-cgm"')

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
46,17-cgm,cgm,10087,10011,2019-01-07,2023-03-13
47,17-cgm,timeseries,10087,10011,2019-01-07,2023-03-13
48,17-cgm,iglu,10087,10011,2019-01-07,2023-03-24
49,17-cgm,iglu_daily,112023,10011,2019-01-07,2023-03-24


In [50]:
df_res.to_csv('s3://ds-users/mariag/pheno_master_data_collection_v1_1_count_converted.csv', index=False)